# CMA - Memory Management

## Introduction to CMA
The Contiguous Memory Allocator (CMA) is a framework allowing machine-specific configurations for
physically-contiguous memory management.

This notebook introduces the reason why CMA is required, and the typical steps to use the CMA on PYNQ-Z1.

## Why is it needed?
Many devices on embedded systems require contiguous blocks of memory to operate. Such devices include cameras, hardware video decoders and encoders, etc. Such devices often require big memory buffers (a full HD frame is, for instance, more then 2 mega pixels large, i.e., more than 6 MB of memory). This makes mechanisms such as `kmalloc()` very ineffective. 

In addition, some embedded devices impose additional requirements on the buffers, e.g., they can operate only on buffers allocated in particular location/memory bank (if system has more than one memory bank) or buffers aligned to a particular memory boundary.

PYNQ-Z1 kernel has support for 128 MB of CMA memory.

## Step 1: Importing the driver

In [1]:
from pynq.drivers import xlnk

## Step 2: Instantiating the memory manager

In [2]:
memmanager = xlnk.xlnk()

## Step 3: Allocating memory

### Allocating non-cacheable memory
The following cell allocates 10 bytes and marks the memory non-cacheable.

In reality, a whole page is allocated.

In [3]:
m1 = memmanager.cma_alloc(10)
memmanager.cma_stats()

{'Buffer Count': 1, 'CMA Memory Available': 133918720, 'CMA Memory Usage': 10}

### Allocating memory for floating-point numbers
The following cell allocates 10 locations for `float` and marks the memory non-cacheable.

In [4]:
m2 = memmanager.cma_alloc(10, data_type = "float")
memmanager.cma_stats()

{'Buffer Count': 2, 'CMA Memory Available': 133914624, 'CMA Memory Usage': 50}

### Allocating memory for 64-bit data
The following cell allocates 128 locations for `long long` and specifies cacheable memory.

Since `long long` is 8 bytes, we get 1024 allocated bytes in total.

In [5]:
m3 = memmanager.cma_alloc(128, cacheable = 1, data_type = "long long")
memmanager.cma_stats()

{'Buffer Count': 3,
 'CMA Memory Available': 133914624,
 'CMA Memory Usage': 1074}

### Casting buffers
A CMA buffer can be cast to any valid C type, e.g., `uint8_t`.

In [6]:
m3 = memmanager.cma_cast(m3, "int")

### Copying between buffers

In [7]:
for i in range(0,256):
    m3[i] = i
print("before copy: ", m2[1])

memmanager.cma_memcopy(m2, m3,10)
print("after copy: ", m2[1])

before copy:  0.0
after copy:  1.401298464324817e-45


### Bytearray and memoryview support

In [8]:
bytes(memmanager.cma_get_buffer(m3,10))

b'\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00'

## Step 4: Freeing the memory
Similar to `alloc()`, buffer has to be freed after use.

In [9]:
memmanager.cma_free(m2)

In [10]:
memmanager.cma_stats()

{'Buffer Count': 2,
 'CMA Memory Available': 133914624,
 'CMA Memory Usage': 1034}

In [11]:
memmanager.cma_free(m1)
memmanager.cma_free(m3)